In [145]:
import glob

import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [146]:
root_dir = "./kfood"
categories = ['Chicken',
              'Dolsotbab',
              'Jeyugbokk-eum',
              'Kimchi',
              'Samgyeobsal',
              'SoybeanPasteStew']

In [147]:
nb_classes = len(categories)  # 6

In [148]:
image_width = 64
image_height = 64

In [149]:
X = []  # 이미지 데이터
Y = []  # 레이블 데이터

In [150]:
for idx, category in enumerate(categories):
    image_dir = root_dir + "/" + category
    files = glob.glob(image_dir + "/" + "*.jpg")
    print(image_dir + "/" + "*.jpg")
    for f in files:
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_width, image_height))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)

X = np.array(X)
Y = np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)
np.save(root_dir + "/kfood.npy", xy)

./kfood/Chicken/*.jpg
./kfood/Dolsotbab/*.jpg
./kfood/Jeyugbokk-eum/*.jpg
./kfood/Kimchi/*.jpg
./kfood/Samgyeobsal/*.jpg
./kfood/SoybeanPasteStew/*.jpg


In [151]:
%pip install keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import np_utils

image_size = 64

Note: you may need to restart the kernel to use updated packages.


In [152]:
# 데이터 로딩
def load_datasets():
    X_train, X_test, y_train, y_test = np.load("./kfood/kfood.npy", allow_pickle=True)
    X_train = X_train.reshape(-1, 64, 64, 1) / 255
    X_test = X_test.reshape(-1, 64, 64, 1) / 255
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return X_train, X_test, y_train, y_test

In [153]:
# 모델 구성
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, padding="same", input_shape=in_shape))
    model.add(Activation("relu"))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(64, 3, 3, padding="same"))
    model.add(Activation("relu"))
    model.add(Convolution2D(64,3,3))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation("softmax"))
    model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    return model

In [154]:
def model_train(x, y):
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=32, epochs=30)
    return model

In [155]:
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print("loss = ", score[0])
    print("accuracy = ", score[1])

In [156]:
X_train, X_test, y_train, y_test = load_datasets()
model = model_train(X_train, y_train)
model_eval(model, X_test, y_test)

model.save("./kfood/kfood_model.h5")

ValueError: Data cardinality is ambiguous:
  x sizes: 399
  y sizes: 133
Make sure all arrays contain the same number of samples.